## Importing libs

In [60]:
import time
import numpy as np
import pandas as pd
import pickle

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import json

### Set up selenium browser

In [61]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('headless')                        
driver = webdriver.Chrome(executable_path='./chromedriver', options=options)

## Web scraping TASS

In [62]:
def get_content(driver: webdriver, api: str) -> json:
    '''Open a link by selenium webdriver, scrap the page and returns result with proposed news in json format'''
    driver.get(api) # открываем ссылку
    time.sleep(3) # даем прогрузиться
    
    page_source = driver.page_source # берем страницу
    soup = BeautifulSoup(page_source, 'html.parser') # парсим
    data = json.loads(soup.body.text) # достаем текст и превращаем в формат json
    
    return data


def update_news(news: list, data: json):
    '''Download news from Tass ural (/ural-news) and add to the existing list of news
    '''
    new_news = [] # список всех распарсенных новостей
    for item in data['result']: # идем по каждой новости
        cur_news = {} 
        cur_news['composite_id'] = item['composite_id']
        cur_news['title'] = item['title']
        cur_news['subtitle'] = item['subtitle']
        cur_news['lead'] = item['lead']
        cur_news['url'] = 'https://tass.ru' + item['url']
        cur_news['rubrics'] = []
        for rubric in item['rubrics']:
            cur_news['rubrics'].append(rubric['name'])
        cur_news['tags'] = []
        for tag in item['tags']:
            cur_news['tags'].append(tag['name'])
        cur_news['es_updated_dt'] = item['es_updated_dt']
        cur_news['published_date'] = item['es_updated_dt'].split('T')[0]
        new_news.append(cur_news)
        
    news.extend(new_news) # расширяем существующий список новыми новостями
    pass

In [63]:
# загружаем новости со стартовой страницы
api = 'https://tass.ru/tbp/api/v1/content?limit=18&offset=0&lang=ru&rubrics=/ural-news&sort=-es_updated_dt'
data = get_content(driver, api)

In [64]:
data['result'][0] # так выглядит новость

{'id': 14975709,
 'composite_id': '14975709',
 'type': 'news',
 'status': 'published',
 'title': 'QR-коды разместят на объектах культурного наследия главного турмаршрута Тобольска',
 'subtitle': '',
 'lead': 'Коды будут вести на туристический портал города, где можно прочитать информацию о зданиях',
 'url': '/obschestvo/14975709',
 'lang': 'ru',
 'urgency': 'normal',
 'main_media': {'gallery': None, 'image': None, 'video': None},
 'main_rubric_id': 24,
 'rubrics': [{'id': 24,
   'name': 'Общество',
   'url': '/obschestvo',
   'type': 'rubric'},
  {'id': 39,
   'name': 'Туризм и отдых',
   'url': '/turizm-i-otdyh',
   'type': 'rubric'},
  {'id': 4731, 'name': 'Новости Урала', 'url': '/ural-news', 'type': 'rubric'},
  {'id': 4964, 'name': 'Сибирь', 'url': '/sibir-news', 'type': 'rubric'},
  {'id': 6296,
   'name': 'Туризм в России',
   'url': '/turizm-v-rossii',
   'type': 'rubric'}],
 'tags': [{'id': 3929, 'name': 'Россия', 'url': '/rossiya'},
  {'id': 8809, 'name': 'Тюменская область',

In [65]:
news = []
update_news(news, data)

In [66]:
news[0] # так выглядит каждая новость

{'composite_id': '14975709',
 'title': 'QR-коды разместят на объектах культурного наследия главного турмаршрута Тобольска',
 'subtitle': '',
 'lead': 'Коды будут вести на туристический портал города, где можно прочитать информацию о зданиях',
 'url': 'https://tass.ru/obschestvo/14975709',
 'rubrics': ['Общество',
  'Туризм и отдых',
  'Новости Урала',
  'Сибирь',
  'Туризм в России'],
 'tags': ['Россия', 'Тюменская область'],
 'es_updated_dt': '2022-06-20T12:46:30.090014',
 'published_date': '2022-06-20'}

In [69]:
with open('tass', 'wb') as f:
    pickle.dump(news, f)

In [70]:
pd.DataFrame.from_records(news) # можно сделать датафреймом

,composite_id,title,subtitle,lead,url,rubrics,tags,es_updated_dt,published_date
0,14975709,QR-коды разместят на объектах культурного насл...,,Коды будут вести на туристический портал город...,https://tass.ru/obschestvo/14975709,"[Общество, Туризм и отдых, Новости Урала, Сиби...","[Россия, Тюменская область]",2022-06-20T12:46:30.090014,2022-06-20
1,14974953,Свердловские отделения реабилитации после кови...,,"Для пациентов, которые не нуждаются в медикаме...",https://tass.ru/obschestvo/14974953,"[Общество, Новости Урала, Пандемия COVID-19]","[Россия, Свердловская область, Пандемия COVID-19]",2022-06-20T12:01:09.027099,2022-06-20
2,14974687,Удмуртия заключила на ПМЭФ 10 соглашений и мем...,,В республике планируется создать около 500 нов...,https://tass.ru/ekonomika/14974687,"[Экономика и бизнес, Новости Урала, ПМЭФ-2022]","[Россия, Санкт-Петербург, Удмуртия, ПМЭФ-2022]",2022-06-20T11:50:32.453984,2022-06-20
3,14974133,"Фестиваль ""Коляда-Plays"" на Урале собрал 26 те...",,Среди участников ожидается порядка 400-500 чел...,https://tass.ru/kultura/14974133,"[Культура, Новости Урала, Сибирь]","[Россия, Свердловская область]",2022-06-20T11:15:32.582977,2022-06-20
4,14973951,В ДТП с автобусом в Челябинской области постра...,,Среди пострадавших двое детей,https://tass.ru/proisshestviya/14973951,"[Происшествия, Новости Урала]","[Россия, Челябинская область]",2022-06-20T11:00:58.339054,2022-06-20
5,14973665,Энергоблок №4 Белоярской АЭС вывели в плановый...,,После окончания работ объект будет работать на...,https://tass.ru/ekonomika/14973665,"[Экономика и бизнес, Новости Урала]","[Россия, Свердловская область]",2022-06-20T10:44:11.490726,2022-06-20
6,14973513,На Урале задержали депутата гордумы за вымогат...,,Андрею Дедюхину уже предъявлено обвинение,https://tass.ru/proisshestviya/14973513,"[Происшествия, Новости Урала]","[Россия, Свердловская область]",2022-06-20T10:39:04.776300,2022-06-20
7,14973415,УрФУ начнет обучение в новом центре русского я...,,Ежегодно в центре будут учиться 80 студентов,https://tass.ru/obschestvo/14973415,"[Общество, Новости Урала]","[Россия, Китай, Свердловская область]",2022-06-20T10:21:44.626341,2022-06-20
8,14973337,Кандидат от СРЗП подал документы в избирком на...,,Голосование состоится в сентябре,https://tass.ru/politika/14973337,"[Политика, Новости Урала]","[Россия, ""Справедливая Россия - За правду"", Св...",2022-06-20T10:19:13.989152,2022-06-20
9,14971095,"В Екатеринбурге возбудили дело после того, как...",,"По данным следствия, отец был пьян",https://tass.ru/proisshestviya/14971095,"[Происшествия, Новости Урала]","[Россия, Свердловская область]",2022-06-20T07:12:11.030087,2022-06-20


## Downloading past news

In [71]:
with open('tass', 'rb') as f:
    news = pickle.load(f)

In [77]:
i = 0
while True:
    time.sleep(5)
    i += 1
    last_es_updated_dt = news[-1]['es_updated_dt']
    iter_api = f'https://tass.ru/tbp/api/v1/content?limit=15&last_es_updated_dt={last_es_updated_dt}&lang=ru&rubrics=/ural-news&sort=-es_updated_dt'
    data = get_content(driver, iter_api)
    update_news(news, data)
    if i == 100:
        with open('tass', 'wb') as f:
            pickle.dump(news, f)

KeyboardInterrupt: 

In [164]:
driver.close()

In [ ]:
with open('tass', 'wb') as f:
    pickle.dump(news, f)